# Six Degrees of Kevin Bacon

![](images/Kevin_Bacon.jpg)

This activity is motivated by the text **Web Scraping with Python** by Ryan Mitchell, available through O'Reilly [here](http://shop.oreilly.com/product/0636920078067.do).  This book goes in depth with much more on using different libraries with Python around common webscraping tasks and I highly recommend it.  We will focus on the activity of moving from a base page to further pages through their links.  

In [18]:
import requests
from bs4 import BeautifulSoup
import re

### Scraping Links

Below, we take the page dealing with the six degrees of Keving Bacon problem.  Here, our goal is to extract links to other pages that we will subsequently pass to requests.  Recall that a link is located in an `<a>` tag and the link is contained in the `href` attribute.  For example, the tag

```HTML
<a href="/wiki/Six_degrees_of_separation" title="Six degrees of separation">six degrees of separation</a>
```

references the Six Degrees of Separation article.  Note that this is a url within Wikipedia.  We can isolate these inner Wikipedia references.  To begin, let's inspect the link content.

In [34]:
response = requests.get('https://en.wikipedia.org/wiki/Six_Degrees_of_Kevin_Bacon')

In [35]:
soup = BeautifulSoup(response.text, 'html.parser')

In [36]:
soup.find('a')

<a id="top"></a>

In [37]:
soup.find_all('a')[:10]

[<a id="top"></a>,
 <a class="mw-jump-link" href="#mw-head">Jump to navigation</a>,
 <a class="mw-jump-link" href="#p-search">Jump to search</a>,
 <a class="image" href="/wiki/File:Kevin_Bacon.jpg"><img alt="" class="thumbimage" data-file-height="461" data-file-width="369" height="275" src="//upload.wikimedia.org/wikipedia/commons/thumb/d/d2/Kevin_Bacon.jpg/220px-Kevin_Bacon.jpg" srcset="//upload.wikimedia.org/wikipedia/commons/thumb/d/d2/Kevin_Bacon.jpg/330px-Kevin_Bacon.jpg 1.5x, //upload.wikimedia.org/wikipedia/commons/d/d2/Kevin_Bacon.jpg 2x" width="220"/></a>,
 <a class="internal" href="/wiki/File:Kevin_Bacon.jpg" title="Enlarge"></a>,
 <a class="mw-redirect" href="/wiki/Parlor_game" title="Parlor game">parlor game</a>,
 <a href="/wiki/Six_degrees_of_separation" title="Six degrees of separation">six degrees of separation</a>,
 <a href="/wiki/Kevin_Bacon" title="Kevin Bacon">Kevin Bacon</a>,
 <a href="/wiki/Hollywood" title="Hollywood">Hollywood</a>,
 <a href="/wiki/Kevin_Bacon" ti

In [23]:
for link in soup.find_all('a')[:10]:
    print(link.attrs)

{'id': 'top'}
{'class': ['mw-jump-link'], 'href': '#mw-head'}
{'class': ['mw-jump-link'], 'href': '#p-search'}
{'href': '/wiki/File:Kevin_Bacon.jpg', 'class': ['image']}
{'href': '/wiki/File:Kevin_Bacon.jpg', 'class': ['internal'], 'title': 'Enlarge'}
{'href': '/wiki/Parlor_game', 'class': ['mw-redirect'], 'title': 'Parlor game'}
{'href': '/wiki/Six_degrees_of_separation', 'title': 'Six degrees of separation'}
{'href': '/wiki/Kevin_Bacon', 'title': 'Kevin Bacon'}
{'href': '/wiki/Hollywood', 'title': 'Hollywood'}
{'href': '/wiki/Kevin_Bacon', 'title': 'Kevin Bacon'}


In [24]:
for link in soup.find_all('a')[:10]:
    if 'href' in link.attrs:
        print(link.attrs['href'])

#mw-head
#p-search
/wiki/File:Kevin_Bacon.jpg
/wiki/File:Kevin_Bacon.jpg
/wiki/Parlor_game
/wiki/Six_degrees_of_separation
/wiki/Kevin_Bacon
/wiki/Hollywood
/wiki/Kevin_Bacon


Okay, seems there are links outside of the inner wiki links.  However, we see that the wiki links contain `/wiki/`, no colons, and the links are all within the body of the page.  Exploiting these means we can write a regular expression 

```
^(/wiki/)((?!:).)*$
```

that will match only the wiki links.  

In [38]:
for link in soup.find_all('a')[:10]:
    linkers = re.compile('^(/wiki/)')

In [39]:
linkers

re.compile(r'^(/wiki/)', re.UNICODE)

In [40]:
for link in soup.find('div', {'id': 'bodyContent'}).find_all('a', href = re.compile('^(/wiki/)((?!:).)*$'))[:10]:
    if 'href' in link.attrs:
        print(link.attrs['href'])

/wiki/Parlor_game
/wiki/Six_degrees_of_separation
/wiki/Kevin_Bacon
/wiki/Hollywood
/wiki/Kevin_Bacon
/wiki/Kevin_Bacon
/wiki/Charitable_organization
/wiki/SixDegrees.org
/wiki/Premiere_(magazine)
/wiki/Kevin_Bacon


### A Function for Links

Now, let's write a function that extracts the link from any wikipedia page.  We should be able to use the idea that the links we care about are located in the same place as our Six Degrees example.  

In [41]:
def get_wikilinks(url):
    links = []
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    for link in soup.find('div', {'id': 'bodyContent'}).find_all('a', href = re.compile('^(/wiki/)((?!:).)*$')):
        links.append(link)
    return links

In [42]:
links = get_wikilinks('https://en.wikipedia.org/wiki/Kevin_Bacon')

In [43]:
links[:10]

[<a class="mw-disambig" href="/wiki/Kevin_Bacon_(disambiguation)" title="Kevin Bacon (disambiguation)">Kevin Bacon (disambiguation)</a>,
 <a href="/wiki/Philadelphia" title="Philadelphia">Philadelphia</a>,
 <a href="/wiki/Pennsylvania" title="Pennsylvania">Pennsylvania</a>,
 <a href="/wiki/Kyra_Sedgwick" title="Kyra Sedgwick">Kyra Sedgwick</a>,
 <a href="/wiki/Sosie_Bacon" title="Sosie Bacon">Sosie Bacon</a>,
 <a href="/wiki/Edmund_Bacon_(architect)" title="Edmund Bacon (architect)">Edmund Bacon</a>,
 <a href="/wiki/Michael_Bacon_(musician)" title="Michael Bacon (musician)">Michael Bacon</a>,
 <a href="/wiki/Footloose_(1984_film)" title="Footloose (1984 film)">Footloose</a>,
 <a href="/wiki/JFK_(film)" title="JFK (film)">JFK</a>,
 <a href="/wiki/A_Few_Good_Men" title="A Few Good Men">A Few Good Men</a>]

In [44]:
for i in links[:10]:
    print(type(i))

<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>


### Connecting Pages

Now, we want to follow these references, gather more urls, and repeat. For the sake of not running to exhaustion, I abbreviate the output using only a large length requirement for the link list.  To traverse all the pages we would simply change the 

```python
while len(links) > 100:
```

to 

```python
while len(links) > 0:
```

In [45]:
import random

def get_wikilinks(url):
    links = []
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    for link in soup.find('div', {'id': 'bodyContent'}).find_all('a', href = re.compile('^(/wiki/)((?!:).)*$')):
        links.append(link)
    return links

In [46]:
links = get_wikilinks('https://en.wikipedia.org/wiki/Kevin_Bacon')
while len(links) > 100:
    newArticle = 'https://en.wikipedia.org' + links[random.randint(0, len(links)-1)].attrs['href']
    print(newArticle)
    links = get_wikilinks(newArticle)

https://en.wikipedia.org/wiki/I_Love_Dick_(TV_series)
https://en.wikipedia.org/wiki/Ripper_Street
https://en.wikipedia.org/wiki/Arrested_Development_(TV_series)
https://en.wikipedia.org/wiki/The_Last_Kingdom_(TV_series)
https://en.wikipedia.org/wiki/Matthew_MacFadyen
https://en.wikipedia.org/wiki/BBC_Radio_4
https://en.wikipedia.org/wiki/CBS_Drama
https://en.wikipedia.org/wiki/KPIX-TV
https://en.wikipedia.org/wiki/1964_Chicago_Cubs_season
https://en.wikipedia.org/wiki/WGN-TV
https://en.wikipedia.org/wiki/Ryne_Sandberg#"The_Sandberg_Game"
https://en.wikipedia.org/wiki/Rob_Neyer


### Problem

Write a function to retrieve a list of albums of any area you are interested in using Wikipedia's list of list of albums page: https://en.wikipedia.org/wiki/Lists_of_albums.

In [60]:
response = requests.get('https://en.wikipedia.org/wiki/Walt_Disney_Records_discography')

In [61]:
soup = BeautifulSoup(response.text, 'html.parser')

In [62]:
soup.find('table')

<table class="wikitable">
<tbody><tr>
<th>Year</th>
<th>Soundtrack</th>
<th>Original release</th>
<th>Reissue
</th></tr>
<tr>
<td>1937</td>
<td><i><a href="/wiki/Snow_White_and_the_Seven_Dwarfs_(soundtrack)" title="Snow White and the Seven Dwarfs (soundtrack)">Snow White and the Seven Dwarfs</a></i></td>
<td>January 1938 (three 78rpm singles) <a class="mw-redirect" href="/wiki/Victor_Records" title="Victor Records">Victor Records</a></td>
<td>June 8, 1993 (CD)<br/>September 25, 2001 (CD) (Remastered)
</td></tr>
</tbody></table>

In [63]:
tables = soup.find_all('table')

In [65]:
tables[0]

<table class="wikitable">
<tbody><tr>
<th>Year</th>
<th>Soundtrack</th>
<th>Original release</th>
<th>Reissue
</th></tr>
<tr>
<td>1937</td>
<td><i><a href="/wiki/Snow_White_and_the_Seven_Dwarfs_(soundtrack)" title="Snow White and the Seven Dwarfs (soundtrack)">Snow White and the Seven Dwarfs</a></i></td>
<td>January 1938 (three 78rpm singles) <a class="mw-redirect" href="/wiki/Victor_Records" title="Victor Records">Victor Records</a></td>
<td>June 8, 1993 (CD)<br/>September 25, 2001 (CD) (Remastered)
</td></tr>
</tbody></table>

In [66]:
tables[0].find('td')

<td>1937</td>

In [73]:
tables[0].find_all('td')

[<td>1937</td>,
 <td><i><a href="/wiki/Snow_White_and_the_Seven_Dwarfs_(soundtrack)" title="Snow White and the Seven Dwarfs (soundtrack)">Snow White and the Seven Dwarfs</a></i></td>,
 <td>January 1938 (three 78rpm singles) <a class="mw-redirect" href="/wiki/Victor_Records" title="Victor Records">Victor Records</a></td>,
 <td>June 8, 1993 (CD)<br/>September 25, 2001 (CD) (Remastered)
 </td>]

In [74]:
year = []
title = []
issue = []
reissue = []

In [75]:
for ent in tables:
    labels=ent.find_all('td')
    year.append(labels[0])
    title.append(labels[1])
    issue.append(labels[2])
    reissue.append(labels[3])

In [77]:
title

[<td><i><a href="/wiki/Snow_White_and_the_Seven_Dwarfs_(soundtrack)" title="Snow White and the Seven Dwarfs (soundtrack)">Snow White and the Seven Dwarfs</a></i></td>,
 <td><i><a href="/wiki/Pinocchio_(soundtrack)" title="Pinocchio (soundtrack)">Pinocchio</a></i></td>,
 <td><span data-sort-value="000000001981-01-01-0000" style="white-space:nowrap">1981</span></td>,
 <td><span data-sort-value="000000002010-03-02-0000" style="white-space:nowrap">March 2, 2010</span>
 </td>]